In [ ]:
import cvxpy as cp
import numpy as np
import scipy.sparse as sp

# Placeholder for loading your dataset
# For example, Demand could be loaded or defined here
# Demand = np.load('path_to_your_demand_data.npy')

# Cost parameters
b = 30
h = 20
r = b / (b + h)

# Data sizes
lntr = 12 * 7 * 16  # Training data size
lnva = lntr // 2  # Validation data size
lnte = lnva  # Test data size

# Sparse matrices to record results
QSAA = sp.dok_matrix((lnte, 1))  # q0 factors
CostSAA = sp.dok_matrix((lnte, 1))  # In-sample cost
TestSAA = sp.dok_matrix((lnte, 1))  # Test cost

for t in range(lntr + lnva + 1, lntr + lnva + lnte + 1):
    print(t)
    q0 = cp.Variable()
    C = cp.Variable(lntr)
    dum1 = cp.Variable(lntr)
    dum2 = cp.Variable(lntr)
    
    # Define the objective function and constraints for the optimization problem
    objective = cp.Minimize(cp.sum(C) / lntr)
    constraints = []
    for i in range(lntr):
        demand_i = Demand[t - lntr + i - 1]  # Placeholder for demand data
        constraints += [
            C[i] == (b + h) * (r * dum1[i] + (1 - r) * dum2[i]),
            dum1[i] >= 0,
            dum1[i] >= demand_i - q0,
            dum2[i] >= 0,
            dum2[i] >= q0 - demand_i,
        ]
    constraints += [q0 >= 0]
    
    # Solve the optimization problem
    prob = cp.Problem(objective, constraints)
    prob.solve()

    # Record the results
    QSAA[t - lntr - lnva, 0] = q0.value
    CostSAA[t - lntr - lnva, 0] = prob.value
    # TestSAA[t - lntr - lnva, 0] = nvcost(q0.value, Demand[t], b, h)  # Placeholder for nvcost function

# Replace 'nvcost' with your actual cost function calculation and adjust the data loading part as per your dataset.
